# notebook for slicing the input images

In [1]:
# What version of Python do you have?
import sys
import numpy as np
import math
from PIL import Image, ImageOps
from numpy import asarray
import matplotlib.pyplot as plt
import os
from os import walk

import pandas as pd

from tqdm import tqdm

import gc
import copy
import cv2

# import sklearn as sk
from pathlib import Path
import plotly.graph_objects as go
from tensorflow.keras.preprocessing.image import load_img

In [2]:
#cropping of images in numpy array
def cropNpArrayIm(npArray,h_w): #sqare crop h = w = h_w
    outArray = np.empty((0,npArray.shape[1],h_w,3));
    #here a for is used to take each image of the array and crop using the numpy function array_split on each axis
    for im in npArray:
        # imageCropX = np.array(np.array_split(im,int(npArray.shape[1]/h_w),axis=0));             #the image is split in (shape[1]/h_w) parts horizontally
        imageCropY = np.array(np.array_split(im,int(npArray.shape[2]/h_w),axis=1));     #the images resulted from the firts split are split in (shape[2]/h_w) parts vertically
        # imageCropY = np.concatenate(imageCropY, axis=0)                                         #the cropped images are concatenated to result a single array
        outArray = np.concatenate((outArray,imageCropY),axis=0)                                 #the out array is fulled by the images
    return outArray

In [3]:
def load_crop_im(pt,dirOut,w, resize, outSize = [256,256]):
    if resize:
        img = load_img(pt,target_size=(outSize[0],outSize[1]))
    else:
        img = Image.open(pt)

    img = asarray(img)
    cropNumber = 0
    imageCropY = np.array_split(img,int(img.shape[1]/w),axis=1);     #the images resulted from the firts split are split in (shape[2]/h_w) parts vertically
    for im in imageCropY:
        filename = pt.stem
        filename = filename + '_' + str(cropNumber) + pt.suffix
        im = Image.fromarray(im)
        im.save(dirOut / filename)
        cropNumber = int(cropNumber) + 1

In [4]:
pathImagesIn = Path('../DB/train_images_masks/')
pathImagesOut = Path('../DB/cropped_train_images_masks/')

In [5]:
pathImagesOut.mkdir(parents=True, exist_ok=True)

In [45]:
f = []
d = ''
for (dirpath,dirnames,filenames) in walk(pathImagesIn):
    f.extend(filenames)
    d = Path(dirpath)

f = [d / Path(el) for el in f]

In [46]:
w = 160
for pt in f:
    load_crop_im(pt,pathImagesOut,w,False)

# create db excell

In [6]:
cvsPath = Path('../DB/train.csv')
SevDB = pd.read_csv(cvsPath)

In [7]:
SevDB.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


## create new db

In [8]:
filenames = []
for (dirpath,dirnames,filenames) in walk(pathImagesOut):
    filenames.extend(filenames)

In [9]:
filenames = [Path(el).name for el in filenames]

In [10]:
filenames

['0002cc93b_0.jpg',
 '0002cc93b_1.jpg',
 '0002cc93b_2.jpg',
 '0002cc93b_3.jpg',
 '0002cc93b_4.jpg',
 '0002cc93b_5.jpg',
 '0002cc93b_6.jpg',
 '0002cc93b_7.jpg',
 '0002cc93b_8.jpg',
 '0002cc93b_9.jpg',
 '0007a71bf_0.jpg',
 '0007a71bf_1.jpg',
 '0007a71bf_2.jpg',
 '0007a71bf_3.jpg',
 '0007a71bf_4.jpg',
 '0007a71bf_5.jpg',
 '0007a71bf_6.jpg',
 '0007a71bf_7.jpg',
 '0007a71bf_8.jpg',
 '0007a71bf_9.jpg',
 '000a4bcdd_0.jpg',
 '000a4bcdd_1.jpg',
 '000a4bcdd_2.jpg',
 '000a4bcdd_3.jpg',
 '000a4bcdd_4.jpg',
 '000a4bcdd_5.jpg',
 '000a4bcdd_6.jpg',
 '000a4bcdd_7.jpg',
 '000a4bcdd_8.jpg',
 '000a4bcdd_9.jpg',
 '000f6bf48_0.jpg',
 '000f6bf48_1.jpg',
 '000f6bf48_2.jpg',
 '000f6bf48_3.jpg',
 '000f6bf48_4.jpg',
 '000f6bf48_5.jpg',
 '000f6bf48_6.jpg',
 '000f6bf48_7.jpg',
 '000f6bf48_8.jpg',
 '000f6bf48_9.jpg',
 '0014fce06_0.jpg',
 '0014fce06_1.jpg',
 '0014fce06_2.jpg',
 '0014fce06_3.jpg',
 '0014fce06_4.jpg',
 '0014fce06_5.jpg',
 '0014fce06_6.jpg',
 '0014fce06_7.jpg',
 '0014fce06_8.jpg',
 '0014fce06_9.jpg',


In [14]:
#find if there is an error and which kind

defects = []
kindDefects = []


# i = 0
for pt in filenames:
    fullPt = pathImagesOut / Path(pt)
    img = Image.open(fullPt)
    img = asarray(img)
    difct = np.any(img > 0.5)
    defects.append(difct)

    if difct:
        oldName = pt.split('_')[0]
        kdifct = SevDB.loc[SevDB['ImageId'].str.contains(str(oldName), case=False)]['ClassId'].values.tolist()
        # print(kdifct)
        kindDefects.append(kdifct[0])
    else:
        kdifct = 0
        kindDefects.append(kdifct)
    # print(difct)

    # if i> 5:
    #     break
    # i = i +1

In [15]:
croppedDB = pd.DataFrame(list(zip(filenames,defects,kindDefects)),columns=['PathId','Has_Defect','Kind_Defect'])

In [16]:
croppedDB.to_csv('../DB/cropped_train.csv')

In [ ]:
croppedDB

In [10]:
def load_crop_im_vert(pt,dirOut,w, resize = False, outSize = [256,256]):
    if resize:
        img = load_img(pt,target_size=(outSize[0],outSize[1]))
    else:
        img = Image.open(pt)

    img = asarray(img)
    cropNumber = 0
    print(img.shape)
    imageCropY = np.array_split(img,int(img.shape[0]/w),axis=0);     #the images resulted from the firts split are split in (shape[2]/h_w) parts vertically
    for im in imageCropY:
        filename = pt.stem
        filename = filename + '_' + str(cropNumber) + pt.suffix
        im = Image.fromarray(im)
        im.save(dirOut / filename)
        cropNumber = int(cropNumber) + 1


def cropAllImagesInPath(path, outpath, outSize_W, resize = False):
    imgs = []
    paths = []
    for (dirpath, dirnames, filenames) in walk(path):
        paths.extend(filenames)
        break

    for p in tqdm(paths):
        p = Path(p)
        # print(p)
        # fullPath = Path(str(basePath) + str(p))
        fullPath = path / p
        # print(basePath)
        # print(fullPath)
        load_crop_im_vert(fullPath,outpath,outSize_W,resize = resize)
    return imgs

In [5]:
vsXpath = Path('../DB/Video-Systems/labelled/db/X/')
vsYpath = Path('../DB/Video-Systems/labelled/db/Y/')

vsXpathCropped = Path('../DB/Video-Systems/labelled/db/Cropped/X/')
vsYpathCropped = Path('../DB/Video-Systems/labelled/db/Cropped/Y/')

In [6]:
vsXpathCropped.mkdir(parents=True, exist_ok=True)
vsYpathCropped.mkdir(parents=True, exist_ok=True)

In [11]:
VideSystemDB_X = cropAllImagesInPath(vsXpath,vsXpathCropped,157)
VideSystemDB_Y = cropAllImagesInPath(vsYpath,vsYpathCropped,157)

  0%|          | 0/80 [00:00<?, ?it/s]

(1256, 1440, 3)


  2%|▎         | 2/80 [00:00<00:18,  4.15it/s]

(1256, 1440, 3)
(1256, 1440, 3)


  4%|▍         | 3/80 [00:00<00:18,  4.17it/s]

...

 96%|█████████▋| 77/80 [00:03<00:00, 23.49it/s]

(1256, 1440, 3)
(1256, 1440, 3)
(1256, 1440, 3)
(1256, 1440, 3)
(1256, 1440, 3)


100%|██████████| 80/80 [00:03<00:00, 23.07it/s]

(1256, 1440, 3)
(1256, 1440, 3)
(1256, 1440, 3)


AttributeError: 'list' object has no attribute 'shape'